# Noisy Circuits


Author: Alexandra Semposki

Maintainer: Alexandra Semposki

Now that VQE has been extensively covered for a noiseless system, we can begin to consider what happens when we add noise into the circuit. Of course, some questions one would have are how the noise occurs in the circuit, how does it affect the circuit results, and how would one correct such errors? In this notebook, we'll cover all of these questions and end with an error mitigation technique that will become very useful for the calculations in the rest of this Jupyter Book.

## On the origin of noise

If you've only ever run a quantum circuit on a simulator, you've probably never encountered errors on your qubits (unless you were using a noisy backend, of course). However, if you run your circuit(s) on a real quantum device, like IBM's quantum computers, you'll notice (once your job stops running) that you don't have just the results you thought you would---instead, you'll have some probabilities in bins you didn't expect at all. This is an example of this noise we're talking about. There are two clear places errors can occur that you'll probably be able to guess quite quickly:

- Qubits

- Quantum gates

Both of these hardware components are highly susceptible to error. Some of the general types of error are described below. 

- __Decoherence__: a qubit will eventually interact with its environment in a real quantum device, which leads to the qubit entangling with this environment, which in turn leads to the qubit altering from its original state into this entangled state. 


- __Dephasing__: this term describes the process of a qubit's phase changing over time; this can be visualised by thinking of a qubit as a vector in the Bloch sphere, and then rotating this vector slowly away from its original position. 


- __Relaxation__: this occurs when a qubit is in a more energetic state, or "excited" state, which we can call $|1\rangle$, and over time it slowly returns back to its "ground" state, which we term $|0\rangle$.


We also can describe some of the specific errors that can occur in a qubit, and which lead to the terms described above. 

- __Bit flips__: a qubit flips from a 0 to a 1 or vice versa; for example, a state may go from $|000\rangle$ to $|010\rangle$ without the action of the quantum gate that would have done this normally. 


- __Phase flips__: a qubit in a superposition state will experience a sign flip between the two components; for example, for a multi-qubit state, the state may go from $(|001\rangle + |110\rangle)$ to $(|001\rangle - |110\rangle)$. Again, this is not a command from the user, it is one that appears to spontaneously occur with no command to do so.


The two error sources above can be corrected for using more qubits to track which qubit's bit or phase flipped---this is called __error correction__. The two types of error control are __error correction__ and __error mitigation__. Error mitigation is not exactly the same thing as error correction; it deals with current practical methods in reducing error in quantum circuits, but not completely removing these errors \[1\]. Error correction tries to do the latter, but often requires hundreds of qubits to perform this task well, which is just not practical at this time for large-scale use. 

## Noiseless vs. noisy circuits: a simple example

Now let's get our hands dirty and build a very simple circuit to demonstrate this noise. We'll use a noiseless simulator first, and then move to a noisy backend to show the difference between simulated quantum circuits and what would happen on a real quantum device. 

In [1]:
#import qiskit and other needed packages
import qiskit

ModuleNotFoundError: No module named 'qiskit'

## References

- \[1\] <tt>mitiq</tt> documentation, "About Error Mitigation," 2020. (Page found [here](https://mitiq.readthedocs.io/en/v.0.1a2/guide/guide_06-error-mitigation.html)).